Hospital data integration for severity prognosis study in COVID-19

Autors: Maria Gabriela Valeriano, Ana Carolina Lorena, Carlos Kiffer

Source: Hospital Beneficência Portuguesa, made available by FAPESP through COVID-19 Data Sharing/BR-USP. The original data is available at https://repositoriodatasharingfapesp.uspdigital.usp.br/

The original data is divided in three files: Pacients, Tests and Outcomes. Each patient has an id and each patient hospital visit receive an new id_attendance as identification. 

In the file tests, there are some cases when the same test have similar but divergent label. This cases were resolved with the help of an expert. The file 'bpsp_exames_01_labels.csv' have this issues corrected. The corrected file and the correspondent codes could be found at github.com/gabivaleriano/defend-prognosis. 


In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.1     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#file: Pacientes

#read the file
patients <- read.csv("bpsp_pacientes_01.csv", sep = "|", stringsAsFactors = FALSE)

#dimensions
patients %>% dim

#check for repeated lines 
patients %>% unique %>% dim


[1] 39000     7

[1] 39000     7

In [3]:
#rename columns
names(patients) <- c('id', 'sex', 'birth_year', 'country', 'state', 'city', 'zip_code')

#number of patients
patients %>% select(id) %>% unique %>% nrow

[1] 39000

In [4]:
#file: Tests

#read file
tests <- read.csv("bpsp_exames_01_labels.csv", stringsAsFactors = FALSE)

#dimensions
tests %>% dim

#check for repeated lines 
tests %>% unique %>% dim

#remove repeated lines 
tests %>% unique -> tests

#number of patients 
tests %>% select(id) %>% unique %>% dim



[1] 5339293      10

[1] 5229119      10

[1] 33800     1

In [5]:
#convert commas -> dots
tests <- data.frame(lapply(tests, function(x) {gsub(",", ".", x)}))


In [6]:
#rename columns
names(tests) <- c('id', 'id_attendance', 'test_date', 'place', 'result', 'measurement_unit', 'normal_value', 'test', 'analyte', 'test_analyte')

#number of id_attendance 
tests %>% select(id_attendance) %>% unique %>% dim

[1] 76506     1

In [7]:
#file: Outcomes

#read the file
outcomes <- read.csv("bpsp_desfecho_01.csv", sep = "|", stringsAsFactors = FALSE) 

#dimensions
outcomes %>% dim

#check for repeated lines 
outcomes %>% unique %>% dim

[1] 217991      8

[1] 217991      8

In [8]:
#rename columns 'clinic', 'result', 'measurement_unit', 'normal_value', 'test', 'analyte', 'test_analyte'
names(outcomes) <- c('id', 'id_attendance', 'attendance_date', 'attendance_type', 'id_clinic', 'clinic', 'outcome_date','outcome_patient')

#number of patients and attendances
outcomes %>% select(id) %>% unique %>% dim
outcomes %>% select(id_attendance) %>% unique %>% dim

[1] 39001     1

[1] 217991      1

In [9]:
#when outcome_patient == death, it does not have outcome_date 
#this issue is due to the criteria adopted for anonymization by the institution that provided the data

outcomes %>% 
filter(str_detect(outcome_patient, 'Obito')) %>%  
count(outcome_date)


outcome_date,n
<chr>,<int>
DDMMAA,834


In [10]:
#check the birth_year
patients %>% count(birth_year)

#when birth year == AAAA the year of birth is equal to or before 1930 
#information given by institution that provided the data
patients$birth_year[patients$birth_year == 'AAAA'] <- '1930'

#check the transformation
patients %>% count(birth_year)

birth_year,n
<chr>,<int>
1931,17
1932,30
1933,27
1934,42
1935,42
1936,40
1937,46
1938,37
1939,42


birth_year,n
<chr>,<int>
1930,107
1931,17
1932,30
1933,27
1934,42
1935,42
1936,40
1937,46
1938,37


In [11]:
#transform the colums outcome_date, attendance_date and test_date to date format
outcomes$outcome_date <- as.Date(outcomes$outcome_date, format="%d/%m/%Y")
outcomes$attendance_date<- as.Date(outcomes$attendance_date, format="%d/%m/%Y")
tests$test_date <- as.Date(tests$test_date, format="%d/%m/%Y")

#dimensions
outcomes %>% dim

#create a new column with the number of days between the first attendance and the outcome 
outcomes %>% mutate(outcome_days = (difftime(outcomes$outcome_date, outcomes$attendance_date, unit = "days"))) -> outcomes
outcomes$outcome_days <- as.integer(outcomes$outcome_days)


[1] 217991      8

In [12]:
#check attendance_date
outcomes %>% count(attendance_date)

attendance_date,n
<date>,<int>
2015-09-08,1
2015-09-10,1
2015-09-14,1
2016-12-01,1
2017-01-06,1
2017-01-26,1
2017-02-01,1
2017-02-13,1
2017-02-14,1


In [13]:
#join patients, tests and outcomes
tests %>% left_join(patients) -> tests
tests %>% left_join(outcomes) -> tests

#check how many patients (ids) and attendances (id_attendance) 
tests %>% select(id) %>% unique %>% dim
tests %>% select(id_attendance) %>% unique %>% dim

#create a new column with the number of days between the first attendance and the test
tests %>% mutate(test_days = (difftime(tests$test_date, tests$attendance_date,  unit = "days"))) -> tests
tests$test_days <- as.integer(tests$test_days)

#dimensions
tests %>% dim

Joining, by = "id"

Joining, by = c("id", "id_attendance")



[1] 33800     1

[1] 76506     1

[1] 5229119      24

In [ ]:
tests %>% write_csv('bp_integrated.csv')